In [ ]:
## including libs

!pip install tensorflow

import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
from keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import load_model

In [ ]:
# baixando o dataset

!wget https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip

!unzip cats_and_dogs_filtered.zip

!rm -rf cats_and_dogs_filtered.zip

In [ ]:
### definindo o dataset

## define o tamanho desejado para as imagens

IMAGE_SHAPE = (224, 244)

#define os diretorios para os dados de treinamento e validação

TRAINING_DATA_DIR = 'cats_and_dogs_filtered/train/'
VALID_DATA_DIR='cats_and_dogs_filtered/validation/'
class_names = ['gato', 'cachorro']

In [ ]:
# preparando o objeto para redimensionar os valores dos pixels das imagens entre 0 e 1

datagen= tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255
)

In [ ]:
# criando conjuntos de dados de treinamento e validação a partir dos diretorios

train_generator = datagen.flow_from_directory(
    TRAINING_DATA_DIR,
    shuffle=True, #embaralha os dados de treinamento
    target_size=IMAGE_SHAPE
)

valid_generator = datagen.flow_from_directory(
    VALID_DATA_DIR,
    shuffle=False, #embaralha os dados de treinamento
    target_size=IMAGE_SHAPE#tamanho desejado das imagens
)

In [ ]:
# plotando algumas imagens do conjunto de treinamento

def plot_dataset(dataset, num_images=5):
  batch = dataset.next()

  images = batch[0]
  labels = batch[1]

  random_indices = np.random.choice(len(images), num_images, replace=False)

  for i, idx in enumerate(random_indices):
    plt.subplot(1, num_images, i+1)
    plt.imshow(images[idx])
    plt.title(class_names[np.argmax(labels[idx])])
    plt.axis('off')
  plt.show
plot_dataset(train_generator)

In [ ]:
## montando o modelo

# Tamanho do lote (batch size) O número de amostras de dados processadas em cada iteração de treinamento.
batch_size = 32
# Épocas (epochs) - O número de vezes que o modelo passará por todo o conjunto de treinamento durante o treinamento.
epochs = 10
# Taxa de aprendizado (learning rate) Um hiperparâmetro que controla o quanto o modelo está aprendendo durante o treinamento.
learning_rate= 0.0001

In [ ]:
from keras.src.layers.pooling.max_pooling1d import MaxPooling1D
#define o modelo da rede neural

model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(filters=8, kernel_size=(3,3), activation='relu',
                           input_shape=(224, 224, 3)),#camada de convolução inicial
    tf.keras.layers.MaxPooling2D(pool_size=(2,2), strides=2), #canada de max pooling
    tf.keras.layers.Conv2D(filters=16, kernel_size=(3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2), strides=2),
    tf.keras.layers.Conv2D(filters=32, kernel_size=(3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2), strides=2),
    tf.keras.layers.Flatten(), #camada de achatamento
    tf.keras.layers.Dense(64, activation='relu'), #camada densa
    tf.keras.layers.Dense(2, activation='softmax')#camada de saida

])

In [ ]:
# compile o modelo com otimizador, função de perda e metricas

model.compile(
    optimizer = tf.keras.optimizers.Adam(learning_rate),
    loss=tf.keras.losses.CategoricalCrossentropy(),
    metrics=['accuracy']
)

print(model.summary())

In [ ]:
# treinamento

# Tamanho do lote (batch size) - O número de amostras de dados processadas em cada iteração de treinamento.
BATCH_SIZE = 32
# Épocas (epochs) O número de vezes que o modelo passará por todo o conjunto de treinamento durante o treinamento.
EPOCHS = 10
# Taxa de aprendizado (learning rate) - Um hiperparâmetro que controla o quanto o modelo está aprendendo durante o treinamento.
learning_rate= 0.0001


In [ ]:
# Definindo o histórico de treinamento para acompanhar o progresso do treinamento
mc = ModelCheckpoint('model.h5', monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)

history = model.fit(train_generator,
                    batch_size=batch_size,
                    epochs=epochs,
                    validation_data=valid_generator,
                    callbacks=[mc],
                    verbose=1)

train_loss = history.history['loss']
train_acc = history.history['accuracy']
valid_loss = history.history['val_loss']
valid_acc = history.history['val_accuracy']


In [ ]:
# Função para plotar as curvas de aprendizado
def save_plots (train_acc, valid_acc, train_loss, valid_loss):
plt.figure(figsize=(12, 9))
plt.plot(train_acc, color='green', linestyle='-', label='train accuracy')
plt.plot(valid_acc, color='blue', linestyle='-',
label='validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.savefig('accuracy.png')
plt.show()

plt.figure(figsize=(12, 9))

/
plt.plot(train_loss, color='orange', linestyle='-' label='train loss') plt.plot(valid_loss, color='red', linestyle='-', label='validation loss') plt.xlabel('Epochs')
143
plt.ylabel('Loss')
144
plt.legend ()
145
146
plt.show()